# Smoke Gets in Your Eyes

This notebook contains the essential final Python code for the results of the _Smoke Gets in Your Eyes_ miniproject.

In [9]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import tree, preprocessing
import sklearn.ensemble as ske
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Preprocess wildfire data for severity prediction

In [10]:
# Read wildfire data from SQLITE database
cnx = sqlite3.connect('FPA_FOD_20170508.sqlite')
df = pd.read_sql_query("SELECT FIRE_YEAR,DISCOVERY_TIME,STAT_CAUSE_DESCR,CONT_DATE,CONT_TIME,LATITUDE,LONGITUDE,STATE,DISCOVERY_DATE,FIRE_SIZE,FIRE_SIZE_CLASS FROM 'Fires'", cnx)
print(df.head())

   FIRE_YEAR DISCOVERY_TIME STAT_CAUSE_DESCR  CONT_DATE CONT_TIME   LATITUDE  \
0       2005           1300    Miscellaneous  2453403.5      1730  40.036944   
1       2004           0845        Lightning  2453137.5      1530  38.933056   
2       2004           1921   Debris Burning  2453156.5      2024  38.984167   
3       2004           1600        Lightning  2453189.5      1400  38.559167   
4       2004           1600        Lightning  2453189.5      1200  38.559167   

    LONGITUDE STATE  DISCOVERY_DATE  FIRE_SIZE FIRE_SIZE_CLASS  
0 -121.005833    CA       2453403.5       0.10               A  
1 -120.404444    CA       2453137.5       0.25               A  
2 -120.735556    CA       2453156.5       0.10               A  
3 -119.913333    CA       2453184.5       0.10               A  
4 -119.933056    CA       2453184.5       0.10               A  


In [14]:
# Convert fire discovery and containment dates to yyyy-mm-dd
df['DISCOVERY_DATE'] = pd.to_datetime(df['DISCOVERY_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
df['CONT_DATE'] = pd.to_datetime(df['CONT_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
print(df.head())

   FIRE_YEAR DISCOVERY_TIME STAT_CAUSE_DESCR  CONT_DATE CONT_TIME   LATITUDE  \
0       2005           1300    Miscellaneous 2005-02-02      1730  40.036944   
1       2004           0845        Lightning 2004-05-12      1530  38.933056   
2       2004           1921   Debris Burning 2004-05-31      2024  38.984167   
3       2004           1600        Lightning 2004-07-03      1400  38.559167   
4       2004           1600        Lightning 2004-07-03      1200  38.559167   

    LONGITUDE STATE DISCOVERY_DATE  FIRE_SIZE FIRE_SIZE_CLASS  
0 -121.005833    CA     2005-02-02       0.10               A  
1 -120.404444    CA     2004-05-12       0.25               A  
2 -120.735556    CA     2004-05-31       0.10               A  
3 -119.913333    CA     2004-06-28       0.10               A  
4 -119.933056    CA     2004-06-28       0.10               A  


In [15]:
# Add new columns for month, date, and weekday for discovery and containment dates

df['DISCOVERY_MONTH'] = pd.DatetimeIndex(df['DISCOVERY_DATE']).month
df['DISCOVERY_DAY'] = pd.DatetimeIndex(df['DISCOVERY_DATE']).day
df['DISCOVERY_DAY_OF_WEEK'] = df['DISCOVERY_DATE'].dt.weekday_name

#df['CONT_DATE'].fillna(0)
df['CONT_MONTH'] = pd.DatetimeIndex(df['CONT_DATE']).month
df['CONT_DAY'] = pd.DatetimeIndex(df['CONT_DATE']).day
df['CONT_DAY_OF_WEEK'] = df['CONT_DATE'].dt.weekday_name
print(df.head())

   FIRE_YEAR DISCOVERY_TIME STAT_CAUSE_DESCR  CONT_DATE CONT_TIME   LATITUDE  \
0       2005           1300    Miscellaneous 2005-02-02      1730  40.036944   
1       2004           0845        Lightning 2004-05-12      1530  38.933056   
2       2004           1921   Debris Burning 2004-05-31      2024  38.984167   
3       2004           1600        Lightning 2004-07-03      1400  38.559167   
4       2004           1600        Lightning 2004-07-03      1200  38.559167   

    LONGITUDE STATE DISCOVERY_DATE  FIRE_SIZE FIRE_SIZE_CLASS  \
0 -121.005833    CA     2005-02-02       0.10               A   
1 -120.404444    CA     2004-05-12       0.25               A   
2 -120.735556    CA     2004-05-31       0.10               A   
3 -119.913333    CA     2004-06-28       0.10               A   
4 -119.933056    CA     2004-06-28       0.10               A   

   DISCOVERY_MONTH  DISCOVERY_DAY DISCOVERY_DAY_OF_WEEK  CONT_MONTH  CONT_DAY  \
0                2              2             W

In [16]:
le = preprocessing.LabelEncoder()
df['STAT_CAUSE_DESCR'] = le.fit_transform(df['STAT_CAUSE_DESCR'])
df['STATE'] = le.fit_transform(df['STATE'])
df['DISCOVERY_DAY_OF_WEEK'] = le.fit_transform(df['DISCOVERY_DAY_OF_WEEK'])
print(df.head())

   FIRE_YEAR DISCOVERY_TIME  STAT_CAUSE_DESCR  CONT_DATE CONT_TIME   LATITUDE  \
0       2005           1300                 7 2005-02-02      1730  40.036944   
1       2004           0845                 6 2004-05-12      1530  38.933056   
2       2004           1921                 3 2004-05-31      2024  38.984167   
3       2004           1600                 6 2004-07-03      1400  38.559167   
4       2004           1600                 6 2004-07-03      1200  38.559167   

    LONGITUDE  STATE DISCOVERY_DATE  FIRE_SIZE FIRE_SIZE_CLASS  \
0 -121.005833      4     2005-02-02       0.10               A   
1 -120.404444      4     2004-05-12       0.25               A   
2 -120.735556      4     2004-05-31       0.10               A   
3 -119.913333      4     2004-06-28       0.10               A   
4 -119.933056      4     2004-06-28       0.10               A   

   DISCOVERY_MONTH  DISCOVERY_DAY  DISCOVERY_DAY_OF_WEEK  CONT_MONTH  \
0                2              2           

In [17]:
df['CONT_DAY_OF_WEEK']=df['CONT_DAY_OF_WEEK'].fillna("Unknown")
df['CONT_DAY']=df['CONT_DAY'].fillna("0")
df['CONT_MONTH']=df['CONT_MONTH'].fillna("0")
df['CONT_TIME']=df['CONT_TIME'].fillna("0")
df['DISCOVERY_TIME']=df['DISCOVERY_TIME'].fillna("0")

df['CONT_DAY_OF_WEEK'] = le.fit_transform(df['CONT_DAY_OF_WEEK'])
df['FIRE_SIZE_CLASS'] = le.fit_transform(df['FIRE_SIZE_CLASS'])
print(df.head())

   FIRE_YEAR DISCOVERY_TIME  STAT_CAUSE_DESCR  CONT_DATE CONT_TIME   LATITUDE  \
0       2005           1300                 7 2005-02-02      1730  40.036944   
1       2004           0845                 6 2004-05-12      1530  38.933056   
2       2004           1921                 3 2004-05-31      2024  38.984167   
3       2004           1600                 6 2004-07-03      1400  38.559167   
4       2004           1600                 6 2004-07-03      1200  38.559167   

    LONGITUDE  STATE DISCOVERY_DATE  FIRE_SIZE  FIRE_SIZE_CLASS  \
0 -121.005833      4     2005-02-02       0.10                0   
1 -120.404444      4     2004-05-12       0.25                0   
2 -120.735556      4     2004-05-31       0.10                0   
3 -119.913333      4     2004-06-28       0.10                0   
4 -119.933056      4     2004-06-28       0.10                0   

   DISCOVERY_MONTH  DISCOVERY_DAY  DISCOVERY_DAY_OF_WEEK CONT_MONTH CONT_DAY  \
0                2            

In [18]:
df['CONT_MONTH']=df['CONT_MONTH'].astype('Float64')
df['CONT_DAY']=df['CONT_DAY'].astype('Float64')

# ML for predicting wildfire severity (size)

In [5]:
import tensorflow
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, BatchNormalization
from keras.optimizers import SGD
from keras.activations import relu
from keras.utils import plot_model

In [19]:
labels=df['FIRE_SIZE_CLASS']
labels.head()

0    0
1    0
2    0
3    0
4    0
Name: FIRE_SIZE_CLASS, dtype: int64

In [20]:
logits=df.drop(['FIRE_SIZE','FIRE_SIZE_CLASS','DISCOVERY_DATE','CONT_DATE','STATE','CONT_TIME','DISCOVERY_TIME'],axis=1)
logits.head()

,FIRE_YEAR,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE,DISCOVERY_MONTH,DISCOVERY_DAY,DISCOVERY_DAY_OF_WEEK,CONT_MONTH,CONT_DAY,CONT_DAY_OF_WEEK
0,2005,7,40.036944,-121.005833,2,2,6,2.0,2.0,7
1,2004,6,38.933056,-120.404444,5,12,6,5.0,12.0,7
2,2004,3,38.984167,-120.735556,5,31,1,5.0,31.0,1
3,2004,6,38.559167,-119.913333,6,28,1,7.0,3.0,2
4,2004,6,38.559167,-119.933056,6,28,1,7.0,3.0,2


In [21]:
labels_cat=keras.utils.to_categorical(labels,7)

In [22]:
x_train, x_test, y_train, y_test = train_test_split(logits,labels_cat,test_size=0.2)

In [ ]:
labels_cat.shape

In [ ]:
model=Sequential()
model.add(BatchNormalization(input_shape=[10]))
model.add(Dense(500,kernel_initializer='truncated_normal'))
model.add(Activation('sigmoid'))
model.add(Dense(300,kernel_initializer='truncated_normal'))
model.add(Activation('sigmoid'))
model.add(Dense(7,kernel_initializer='truncated_normal'))
model.add(Activation('softmax'))

In [ ]:
model.compile(optimizer=SGD(lr=0.01, momentum=0.9,decay=1e-6),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs=10,validation_data=(np.array(x_test),np.array(y_test)))

In [3]:
model=load_model('64%model')

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
a=model.predict(x_test)

In [6]:
plot_model(model, to_file='network.png')

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_67 (Batc (None, 10)                40        
_________________________________________________________________
dense_82 (Dense)             (None, 1000)              11000     
_________________________________________________________________
activation_81 (Activation)   (None, 1000)              0         
_________________________________________________________________
batch_normalization_68 (Batc (None, 1000)              4000      
_________________________________________________________________
dropout_33 (Dropout)         (None, 1000)              0         
_________________________________________________________________
dense_83 (Dense)             (None, 1200)              1201200   
_________________________________________________________________
activation_82 (Activation)   (None, 1200)              0         
__________

# Generating Air quality data

In [ ]:
def preprocess_site_list(site_file):
    """Get the location of air quality measurement sites.
        
    Return the latitude and longitude coordinates in two dictionaries. 
    The keys are site codes as strings and the values are coordinates as floats.
    """

    # Make sure that state, county, and site codes are read as strings
    # instead of numbers
    col_types = {'State Code': str, 'County Code': str, 'Site Number': str}

    sites = pd.read_csv(site_file, dtype=col_types)

    # Columns that can be dropped from the site listing
    cols_to_drop = [
        'Land Use', 'Location Setting', 'Met Site State Code',
        'Met Site County Code', 'Met Site Site Number', 'Met Site Type',
        'Met Site Distance', 'Met Site Direction', 'GMT Offset',
        'Owning Agency', 'Local Site Name', "Address", "Zip Code",
        "State Name", "County Name", "City Name", "CBSA Name", "Tribe Name",
        "Extraction Date"
    ]

    sites = sites.drop(columns=cols_to_drop)

    # Convert site closing dates to datetime64 objects (does not work
    # with read_csv())
    sites['Site Closed Date'] = pd.to_datetime(sites['Site Closed Date'])

    # Get sites that have been closed down before 1992 and drop them
    # from the list
    outdated = sites.loc[
        sites['Site Closed Date'] < pd.to_datetime('01-01-1992')]
    sites = sites.drop(outdated.index)

    # Get the complete site code, add it as a column, and finally
    # use the site code as the index
    sites['site-code'] = sites['State Code'] + '-' + sites[
        'County Code'] + '-' + sites['Site Number']
    sites = sites.set_index('site-code')

    # Divide the data into two Series: one with latitude coordinates and one
    # with longitude coordinates
    sites_lat = sites['Latitude']
    sites_lon = sites['Longitude']

    # Transform the site listings into a dictionary where keys are site
    # codes and values are coordinates}
    sites_lat = sites_lat.to_dict()
    sites_lon = sites_lon.to_dict()

    # Return the two dictionaries with the coordinates
    return sites_lat, sites_lon


def preprocess_aqi(sites_lat, sites_lon, aqi_file):
    """Convert a CSV file with AQI data into a pandas DataFrame.

    Get the following columns from an AQI data file: Date, AQI, Category,
    Defining Parameter, and Defining Site. Ensure that dates are parsed as 
    datetime objects.

    This function also adds coordinate data for the defining site to the data 
    frame.
    """

    # Specify used column names and types to make CSV parsing more efficient
    col_types = {
        'AQI': int,
        'Category': str,
        'Defining Parameter': str,
        'Defining Site': str
    }

    col_names = list(col_types.keys()) + ['Date']

    aqi = pd.read_csv(
        aqi_file, usecols=col_names, dtype=col_types, parse_dates=['Date'])

    # Add column containing the site information (including location
    # coordinates) to each AQI measurement
    aqi['Latitude'] = aqi['Defining Site'].map(sites_lat)
    aqi['Longitude'] = aqi['Defining Site'].map(sites_lon)

    return aqi

In [ ]:
# Get dictionaries containing latitude and longitude coordinates for AQI measurement sites
sites_lat, sites_lon = preprocess_site_list('data/aqi/aqs_sites.csv')

# Generate a list of years 1992–2015
years = [str(y) for y in range(1992, 2016)]

# Generate a dictionary where keys are years (as strings) and values are DataFrames
# containing the AQI data with coordinates added
aqi = {
    y: preprocess_aqi(sites_lat, sites_lon,
                      'data/aqi/daily_aqi_by_county_{}.csv'.format(y))
    for y in years
}


In [ ]:
import math
from pandas.tseries.offsets import DateOffset


LON_AT_EQUATOR = 111
LAT_IN_KM = 111


def get_distance(lat1, lon1, lat2, lon2):
    dist = math.sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2)
    return round(dist, 6)


def dist_as_lon_degree(distance, latitude):
    """A simplified conversion of a specified distance into degrees longitude
    at a specified latitude"""
    lon_deg_as_km = LON_AT_EQUATOR * math.cos(math.radians(latitude))
    return distance / lon_deg_as_km


def get_aqis_during_fire(year, start_date, end_date):
    # Narrow down search by getting measurements from the time of the fire.
    # Note that here the date range is defined as DISCOVERY_DATE + 1 to CONT_DATE + 1:
    # it seems likely that measurements from the discovery date might have been made before
    # the fire was discovered.
    # If either of the given dates is a null value, return an empty dataframe
    if pd.isnull(start_date) or pd.isnull(end_date):
        return pd.DataFrame()
    aqi_df = aqi[year]
    date_range = pd.date_range(start_date + DateOffset(days=1),
                               end_date + DateOffset(days=1))
    return aqi_df.loc[aqi_df['Date'].isin(date_range)]


def get_nearest_measurement(fire_lat, fire_lon, aqi_df):
    # Narrow down the search space by only including measurements that are +/- 0.5 degrees
    # from the fire coordinates (this translates to a square area with the fire in the
    # center: the square is approx. 110km x 110km
    # If the dataframe is empty, return an empty Series
    if aqi_df.empty:
        return pd.Series()
    lon_degrees = dist_as_lon_degree(LAT_IN_KM, fire_lat)
    candidates = aqi_df.loc[(
        (abs(aqi_df['Latitude'] - fire_lat) <= 0.5)
        & (abs(aqi_df['Longitude'] - fire_lon) <= lon_degrees / 2))]
    # If no measurements are within defined range, return an empty Series
    if candidates.empty:
        return pd.Series()
    # Loop through the candidate measurements and get the nearest location
    nearest = np.argmin([
        get_distance(fire_lat, fire_lon, row.Latitude, row.Longitude)
        for row in candidates.itertuples()
    ])
    return candidates.iloc[nearest]

In [ ]:
from datetime import datetime
import sys

# Define the number of rows, the interval at which to save partial results,
# and create arrays for the data
rows = df.shape[0]
cutoff = int(rows / 10)
percent = int(rows / 100)
nearest_measurements = np.array(np.zeros(rows), dtype=object)
aqi_readings = np.array(np.zeros(rows), dtype=np.float32)

# Used to measure time elapsed in running the algorithm
t1 = datetime.now()

# Counter variable used for filenames for partial results
counter = 1

for row in df.itertuples():
    # Write out progress
    if row.Index % percent == 0:
        sys.stdout.write("Progress: {}%  \r".format(
            round((row.Index / rows) * 100, 2)))
        sys.stdout.flush()
    # Get the AQI measurement closest to the fire
    aqi_candidates = get_aqis_during_fire(
        str(row.FIRE_YEAR), row.DISCOVERY_DATE, row.CONT_DATE)
    nearest = get_nearest_measurement(row.LATITUDE, row.LONGITUDE,
                                      aqi_candidates)
    nearest_measurements[row.Index] = nearest.to_dict()
    if nearest.empty:
        aqi_readings[row.Index] = np.nan
    else:
        aqi_readings[row.Index] = nearest['AQI']

    # Save partial results in case script fails to run to the end
    if row.Index > 0 and row.Index % cutoff == 0:
        start = row.Index - cutoff
        stop = row.Index
        subframe = df.iloc[start:stop, :]
        subframe['AQI'] = aqi_readings[start:stop]
        subframe['AIR_QUALITY'] = nearest_measurements[start:stop]
        subframe.to_csv(
            'data/wildfire/wildfires_with_aqi_v2_{}.csv'.format(
                str(counter).zfill(2)))
        counter += 1

# Add the air quality data to the wildfires dataframe, drop rows that do 
# not have an AQI reading (because no nearby measurements were found),
# and save the results into a CSV file
df['AQI'] = aqi_readings
df['AIR_QUALITY'] = nearest_measurements
df.dropna(subset=['AQI'], inplace=True)
df.to_csv('data/wildfire/wildfires_with_aqi_all.csv')

t2 = datetime.now()
delta = t2 - t1

print()
print("Processing {} entries took {} minutes and {} seconds".format(
    rows, delta.seconds // 60, delta.seconds % 60))

# Load in the new created data and delete measurement mistakes

In [84]:
df=pd.read_csv('data/wildfire/wildfires_with_aqi_1.csv')

In [96]:
df=df[df['AQI']<500]

# Plot of fire size and AQI

In [ ]:
aqi=df['AQI'].as_matrix().tolist()

In [ ]:
fire=df['FIRE_SIZE'].as_matrix().tolist()

In [147]:
import bokeh
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.io import show, output_notebook

In [154]:
p = figure(plot_width = 600, plot_height = 600, 
           title = 'Example Glyphs',
           x_axis_label = 'firesize', y_axis_label = 'aqi',
          x_axis_type='log')

In [155]:
p.square(x=fire, y=aqi, size = 1, color = 'black')
show(p)

# linear regression

In [36]:
from sklearn import linear_model

In [158]:
lm = linear_model.LinearRegression()
model = lm.fit(np.array(fire).reshape(-1,1),aqi)

In [162]:
lm.score(np.array(fire).reshape(-1,1),aqi)

0.00020529618176290398

In [163]:
import statsmodels.api as sm

X = fire
#X = X["FIRE_SIZE"]
y = aqi

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     509.7
Date:                Sat, 27 Oct 2018   Prob (F-statistic):          9.17e-113
Time:                        13:56:01   Log-Likelihood:            -1.6156e+06
No. Observations:              289728   AIC:                         3.231e+06
Df Residuals:                  289727   BIC:                         3.231e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0010   4.57e-05     22.577      0.000       0.001       0.001
==============================================================================
Omnibus:                   102089.827   Durbin-Watson:                   0.300
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           406148.491
Skew:                           1.732   Prob(JB):                         0.00
Kurtosis:                       7.652   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Plots

In [ ]:
df['STAT_CAUSE_DESCR'].value_counts().plot(kind='barh',color='coral')
plt.show()

In [ ]:
df_arson = df[df['STAT_CAUSE_DESCR']=='Arson']
df_arson['DAY_OF_WEEK'].value_counts().plot(kind='barh',color='coral')
plt.show()

In [ ]:
def plot_corr(df,size=10):
    corr = df.corr()  #the default method is pearson
    fig, ax = plt.subplots(figsize=(size, size))
    ax.matshow(corr,cmap=plt.cm.Oranges)
    plt.xticks(range(len(corr.columns)), corr.columns)
    plt.yticks(range(len(corr.columns)), corr.columns)
    for tick in ax.get_xticklabels():
        tick.set_rotation(45)    
    plt.show()        
plot_corr(df)

In [ ]:
dfplot=df

In [ ]:
dfplot=dfplot.sort_index(by=['FIRE_SIZE'],axis=0, ascending=False)

In [ ]:
dflol=dfplot[:500]

In [ ]:
dflol.plot(kind='scatter',x='LONGITUDE',y='LATITUDE',color='coral',alpha=0.3)
plt.show()

In [ ]:
df_lightning = df#[df['STAT_CAUSE_DESCR']=='Lightning']
df_lightning['DISCOVERY_DAY_OF_WEEK'].value_counts().plot(kind='barh',color='coral')
plt.show()

In [ ]:
df['DISCOVERY_DAY_OF_WEEK'].value_counts().plot(kind='barh',color='coral')
plt.show()

In [ ]:
df['STATE'].value_counts().head(n=10).plot(kind='barh',color='coral')
plt.show()

In [ ]:
df_CA = df[df['STATE']=='CA']
df_CA['STAT_CAUSE_DESCR'].value_counts().plot(kind='barh',color='coral',title='causes of fires for CA')
plt.show()